In [148]:
from glob import glob 
from fastcore.helpers import load_pickle

import pandas as pd 

In [149]:
all_res = glob('results/20221117_solubility/*')

In [150]:
len(all_res)

115

In [151]:
load_pickle(all_res[0]).keys()

dict_keys(['num_train_points', 'baseline_cm', 'baseline_y_eval', 'modelname', 'predictions', 'completions', 'true', 'cm', 'representation'])

In [152]:
compiled_res = []
baseline_res = []

for res in all_res:
    loaded_res = load_pickle(res)
    if 'representation' in loaded_res: 
        this_res = {
            'representation': loaded_res['representation'],
            'train_set_size': loaded_res['num_train_points'],
            'f1_macro': loaded_res['cm'].F1_Macro,
            'f1_micro': loaded_res['cm'].F1_Micro,
            'acc': loaded_res['cm'].ACC_Macro,
        }
        compiled_res.append(this_res)

        try:
            baseline_res.append({
                'representation': loaded_res['representation'],
                'train_set_size': loaded_res['num_train_points'],
                'f1_macro': loaded_res['baseline_cm'].F1_Macro,
                'f1_micro': loaded_res['baseline_cm'].F1_Micro,
                'acc': loaded_res['baseline_cm'].ACC_Macro,
            })
        except:
            pass

In [153]:
compiled_res = pd.DataFrame(compiled_res)
baseline_res = pd.DataFrame(baseline_res)

In [159]:
compiled_res_grouped = compiled_res.groupby(['representation', 'train_set_size']).agg(['mean', 'std'])

In [160]:
compiled_res_grouped

f1_macro            f1_micro            \
                                   mean       std      mean       std   
representation train_set_size                                           
InChI          10              0.122754  0.041445  0.195111  0.026002   
               50              0.245372  0.052206  0.308857  0.038347   
               500             0.449038  0.037351  0.466571  0.037269   
Name           10              0.146918  0.027312  0.210000  0.022142   
               50              0.158902  0.056365  0.221143  0.032575   
               500             0.361255  0.030149  0.376667  0.031078   
SMILES         10              0.153161  0.057292  0.208667  0.040841   
               50              0.247786  0.031101  0.300000  0.022065   
               500             0.498951  0.009942  0.507714  0.010029   
iupac_names    10              0.103033  0.023959  0.199000  0.017664   
               50              0.174663  0.042504  0.233429  0.040128   
               500             0.397276  0.034305  0.402333  0.029296   
selfies        10              0.123725  0.042860  0.206889  0.035876   
               50              0.200697  0.044704  0.275714  0.053459   
               500             0.446615  0.043775  0.482286  0.026918   

                                    acc            
                                   mean       std  
representation train_set_size                      
InChI          10              0.683807  0.023378  
               50              0.723543  0.015339  
               500             0.786629  0.014907  
Name           10              0.697233  0.025081  
               50              0.688457  0.013030  
               500             0.757311  0.024104  
SMILES         10              0.711107  0.037611  
               50              0.720000  0.008826  
               500             0.803086  0.004011  
iupac_names    10              0.679600  0.007065  
               50              0.693371  0.016051  
               500             0.760933  0.011718  
selfies        10              0.706400  0.029182  
               50              0.710286  0.021384  
               500             0.802838  0.018948

In [166]:
def compile_table_row(row):
    template =  "\\num⁍ {accuracy} \\pm {accuracy_std} ⁌ &  \\num⁍ {f1_micro} \\pm {f1_micro_std}  ⁌ & \\num⁍ {f1_macro} \\pm {f1_macro_std} ⁌\\\\"
    row = row.round(2)
    string = template.format(
        accuracy=row['acc']['mean'],
        accuracy_std=row['acc']['std'],
        f1_micro=row['f1_micro']['mean'],
        f1_micro_std=row['f1_micro']['std'],
        f1_macro=row['f1_macro']['mean'],
        f1_macro_std=row['f1_macro']['std']
    )

    return string.replace("⁍", "{").replace("⁌", "}")


In [192]:
print(compile_table_row(compiled_res_grouped.loc['Name'].loc[500]))

\num{ 0.76 \pm 0.02 } &  \num{ 0.38 \pm 0.03  } & \num{ 0.36 \pm 0.03 }\\


In [177]:
baseline_res_agg= baseline_res.groupby(['train_set_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_69174/4278541108.py:1: FutureWarning: ['representation'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  baseline_res_agg= baseline_res.groupby(['train_set_size']).agg(['mean', 'std'])


In [185]:
print(compile_table_row(baseline_res_agg.loc[500]))

\num{ 0.84 \pm 0.01 } &  \num{ 0.59 \pm 0.01  } & \num{ 0.59 \pm 0.01 }\\
